In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('../database/intents1.json').read()
intents = json.loads(data_file)

for intent in intents['type1']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [6]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-6-7fa144a2511d>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.7, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('../models/model1.h5', hist)

print("model created")

Epoch 1/100
7/7 [==============================] - 0s 1ms/step - loss: 2.2908 - accuracy: 0.0909
Epoch 2/100
7/7 [==============================] - 0s 917us/step - loss: 2.1615 - accuracy: 0.1818
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 2.0931 - accuracy: 0.2727
Epoch 4/100
7/7 [==============================] - 0s 1ms/step - loss: 2.0564 - accuracy: 0.3030
Epoch 5/100
7/7 [==============================] - 0s 1ms/step - loss: 1.9437 - accuracy: 0.2121
Epoch 6/100
7/7 [==============================] - 0s 1ms/step - loss: 1.9089 - accuracy: 0.3939
Epoch 7/100
7/7 [==============================] - 0s 1ms/step - loss: 1.7852 - accuracy: 0.3636
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 1.7000 - accuracy: 0.4545
Epoch 9/100
7/7 [==============================] - 0s 2ms/step - loss: 1.5233 - accuracy: 0.5758
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 1.5755 - accuracy: 0.6061
Epoch 11/100
7/7 [=========

7/7 [==============================] - 0s 1ms/step - loss: 0.4561 - accuracy: 0.8182
Epoch 85/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3935 - accuracy: 0.8485
Epoch 86/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3877 - accuracy: 0.8485
Epoch 87/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3387 - accuracy: 0.8485
Epoch 88/100
7/7 [==============================] - 0s 793us/step - loss: 0.3673 - accuracy: 0.8485
Epoch 89/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4248 - accuracy: 0.8182
Epoch 90/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3423 - accuracy: 0.8788
Epoch 91/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4332 - accuracy: 0.7879
Epoch 92/100
7/7 [==============================] - 0s 872us/step - loss: 0.4235 - accuracy: 0.8182
Epoch 93/100
7/7 [==============================] - 0s 1ms/step - loss: 0.3813 - accuracy: 0.8485
Epoch 94/100
7/7 [===========